Please check out the original [tensorflow implementation](https://github.com/uclmr/emoji2vec) of Emoji2Vec by its authors as well as the [paper](https://arxiv.org/pdf/1609.08359.pdf) for more details. <br>
This notebook is intended to provide an intuition behind Emoji2Vec and provide an idea of its features rather than serve as a in-depth analysis of the framework.

In [2]:
# External dependencies
import os
import numpy as np
import pickle as pk
import gensim.models as gsm
import torch
from tabulate import tabulate

# Internal dependencies
from model import Emoji2Vec, ModelParams
from phrase2vec import Phrase2Vec
from utils import build_kb, get_examples_from_kb, generate_embeddings, get_metrics, generate_predictions

### What is Emoji2Vec?
Emoji2Vec representations are a form of distributed representations for Unicode emojis, trained directly from natural language descriptions of emojis. <br>
This notebook presents a high level walkthrough the process of training Emoji2Vec representations and demonstrates some experimental results. <br><br>
First, lets define the logical steps in training Emoji2Vec representations.
1. For each emoji in a data set a number of natural language descriptions is collected.
2. Each description is encoded to a fixed form vector in a high dimensional space. 
    - Although it can be done by an arbitrary encoding method, this implementation follows the approach presented in the [paper](https://arxiv.org/pdf/1609.08359.pdf) using 300-dimensional [Google News word2vec embeddings](https://code.google.com/archive/p/word2vec/) together with a simple phrase encoder `phrase2vec.Phrase2Vec`.
3. A neural network model is trained to classify emojis from their descriptions.
    - Inside the model each unique emoji has its own vector of parameters that is updated when this emoji is being classified. Through continuous incrementation in training such vectors of parameters become emoji representations.
4. Neural network's parameters are extracted and used as distributed emoji representations that are embedded in the same space as the underlying word embeddings.

### How to train Emoji2Vec representations?
First, we need a couple of things to train Emoji2Vec representations. Most importantly we need something to encode natural language descriptions of emojis to a fixed form high dimensional vectors. For this we are going to use [Google News wor2vec embeddings](https://code.google.com/archive/p/word2vec/), so in order to continue with this notebook make sure you've downloaded those embeddings and placed a `.bin.gz` file in `data/word2vec`. <br><br>
Of course we also need some emojis and their natural language descriptions. Fortunately authors of the paper collected a data set that is part of this repository. <br>
Our training data set is in `data/training`. Let's add this directory to our parameters.

In [3]:
params = {"data_folder": "data/training"}

To train Emoji2Vec representations we need to put emojis in some sort of organised structure, so that after training we know which vector of parameters in the model is associated with which emoji. Of course the same needs to be done for each phrase (natural language description of emoji) in the data set so that we know which descrption describes which emoji.

In [4]:
# Build knowledge base
print('reading training data from: ' + params["data_folder"])
kb, ind2phr, ind2emoji = build_kb(params["data_folder"])


reading training data from: data/training


We'll need this emoji mapping later, so for now let's save it to a file.

In [5]:
params.update({"mapping_file": "emoji_mapping_file.pkl"})

In [6]:
# Save the mapping from index to emoji
pk.dump(ind2emoji, open(params["mapping_file"], 'wb'))

Time to encode descriptions of emojis to fixed form vectors. As mentioned before we're using 300 dimensional Google News embeddings located in `data/word2vec`. As we will need them later we're going to save the generated embeddings of descriptions to a file `phrase_embeddings.pkl`. <br>Let's add those paths to our parameters.

In [7]:
params.update({"word2vec_embeddings_file": "data/word2vec/GoogleNews-vectors-negative300.bin",
               "phrase_embeddings_file": "phrase_embeddings.pkl"})

To reitarete - phrase embeddings are embeddigns of each emoji description that will serve as input for the neural network, whereas Google News embeddings are what we use to generate phrase embeddings. <br>

In [8]:
# Generate embeddings for each phrase in the training set
embeddings_array = generate_embeddings(ind2phr=ind2phr, kb=kb, embeddings_file=params["phrase_embeddings_file"],
                                       word2vec_file=params["word2vec_embeddings_file"])

loading embeddings...


So we have the training input (phrase embeddings), now let's load our training and development data sets.

In [9]:
train_set = get_examples_from_kb(kb=kb, example_type='train')
dev_set = get_examples_from_kb(kb=kb, example_type='dev')

And define training parameters.

In [10]:
model_params = ModelParams(in_dim=300, 
                           out_dim=300, 
                           max_epochs=60, 
                           pos_ex=4, 
                           neg_ratio=1, 
                           learning_rate=0.001,
                           dropout=0.0, 
                           class_threshold=0.5)

Define the model, it needs to know the training parameters, the number of emojis (size of parameter matrix depends on it) and the matrix of phrase embeddings (embeddings of emoji descriptions).

In [11]:
model = Emoji2Vec(model_params=model_params, num_emojis=kb.dim_size(0), embeddings_array=embeddings_array)

Now that the model has been defined, let's train it! 

In [12]:
model.train(kb=kb, epochs=model_params.max_epochs, learning_rate=model_params.learning_rate)

Epoch: 1 
 Training loss: 0.68 
 Training acc: 0.55 
 Training f1: 0.54 
Epoch: 2 
 Training loss: 0.61 
 Training acc: 0.71 
 Training f1: 0.74 
Epoch: 3 
 Training loss: 0.55 
 Training acc: 0.78 
 Training f1: 0.8 
Epoch: 4 
 Training loss: 0.49 
 Training acc: 0.83 
 Training f1: 0.85 
Epoch: 5 
 Training loss: 0.45 
 Training acc: 0.86 
 Training f1: 0.88 
Epoch: 6 
 Training loss: 0.41 
 Training acc: 0.88 
 Training f1: 0.89 
Epoch: 7 
 Training loss: 0.38 
 Training acc: 0.9 
 Training f1: 0.91 
Epoch: 8 
 Training loss: 0.35 
 Training acc: 0.92 
 Training f1: 0.92 
Epoch: 9 
 Training loss: 0.32 
 Training acc: 0.93 
 Training f1: 0.93 
Epoch: 10 
 Training loss: 0.3 
 Training acc: 0.93 
 Training f1: 0.94 
Epoch: 11 
 Training loss: 0.28 
 Training acc: 0.93 
 Training f1: 0.94 
Epoch: 12 
 Training loss: 0.27 
 Training acc: 0.94 
 Training f1: 0.95 
Epoch: 13 
 Training loss: 0.25 
 Training acc: 0.95 
 Training f1: 0.95 
Epoch: 14 
 Training loss: 0.23 
 Training acc: 0.

Let's save the trained PyTorch model to a file so that we can load it later if we need it.

In [13]:
model_folder = 'example_emoji2vec'
if not os.path.isdir(model_folder):
    os.makedirs(model_folder)
    
torch.save(model.nn, model_folder + '/model.pt')

Now that the model is trained and saved we need to extract emoji representations from the neural network's parameters. <br>
This can be done by a `model.Emoji2Vec` method called `create_gensim_files`, which will save the distributed representations of emojis in a format compatible with `gensim.models` allowing us to add Emoji2Vec representations to a gensim model and use them as any other word embeddings.


In [14]:
e2v = model.create_gensim_files(model_folder=model_folder, ind2emoj=ind2emoji, out_dim=model_params.out_dim)

#### Done.
That's it, you've generated your own Emoji2Vec representations which now sit in `example_emoji2vec/` as `emoji2vec.txt` and `emoji2vec.bin`. Good job. <br>
Now is the time for the fun part, for example we can create a gensim model with our newly generated emoji embeddings.

In [15]:
e2v = gsm.KeyedVectors.load_word2vec_format("example_emoji2vec/emoji2vec.bin", binary=True)

Let's take a look at some of the emojis in the data set.

In [16]:
vocabulary = e2v.key_to_index.keys()

In [17]:
# Sample 10 random emojis from the data set.
example_emojis = np.random.choice(list(vocabulary), 10)
print(example_emojis)

['🐶' '👎' '👈' '🗝' '🥣' '🌟' '🇬🇺' '🐲' '🧑\u200d🔬' '⛔']


Each of the above emojis has its own vector representation.

In [18]:
e2v[example_emojis]

array([[ 0.01237914, -0.6340719 , -0.05370076, ...,  0.44213054,
        -0.29079998, -0.02131909],
       [-0.10699613, -0.8837111 ,  0.24367897, ...,  0.06239364,
        -0.40534386, -0.10414707],
       [-0.11828447, -0.1465209 , -0.0148176 , ...,  0.2494605 ,
         0.17834519, -0.19599813],
       ...,
       [ 0.51543856, -0.03214123,  0.05075327, ...,  0.28191367,
        -0.070646  , -1.0734489 ],
       [-0.177938  ,  0.0614969 ,  0.2951721 , ...,  0.15840364,
         0.28126538,  0.2250081 ],
       [-0.53266716, -0.5472906 ,  0.18449265, ...,  0.02622689,
        -0.29174468, -0.28194213]], dtype=float32)

Thanks to the fact that the learnt emoji representations are compatible with `gensim.models`, we can do a lot of cool things like finding similar emojis.

In [19]:
e2v.most_similar('🦎')

[('🦖', 0.5381361842155457),
 ('🐦', 0.5250487923622131),
 ('🐁', 0.5234279632568359),
 ('🦜', 0.517848014831543),
 ('🐭', 0.5172520279884338),
 ('🐟', 0.5159448385238647),
 ('🕋', 0.515628457069397),
 ('🐸', 0.5142031311988831),
 ('🐍', 0.5135039687156677),
 ('🐊', 0.5085179805755615)]

In [20]:
e2v.most_similar('😿')

[('😢', 0.7393065690994263),
 ('😓', 0.606322705745697),
 ('😞', 0.5818456411361694),
 ('😥', 0.5702885389328003),
 ('🙀', 0.5624997019767761),
 ('😔', 0.5577625632286072),
 ('😭', 0.5569652915000916),
 ('😕', 0.537689208984375),
 ('🤧', 0.5143035650253296),
 ('😾', 0.5132414102554321)]

In [21]:
e2v.most_similar('🦥')

[('🐯', 0.6086215376853943),
 ('🦽', 0.596916913986206),
 ('🐄', 0.5850686430931091),
 ('🐮', 0.5691218972206116),
 ('🐷', 0.5652249455451965),
 ('🐌', 0.5475101470947266),
 ('🐕', 0.5405159592628479),
 ('🐁', 0.5357159376144409),
 ('🦊', 0.531782865524292),
 ('🐽', 0.5238668322563171)]

In [22]:
e2v.most_similar('👛')

[('💰', 0.6425021290779114),
 ('💳', 0.6419100761413574),
 ('🛍', 0.5991641283035278),
 ('🏬', 0.5774742364883423),
 ('💵', 0.5749498009681702),
 ('🛒', 0.5715884566307068),
 ('🧾', 0.5660741925239563),
 ('🧳', 0.5629919171333313),
 ('🔷', 0.5309175848960876),
 ('💲', 0.5262939929962158)]

However the real fun starts when we combine the power of emoji embeddings with word embeddings. <br>
Let's create a model that combines the two, this will allow us to measure similarity between emojis and phrases. 

In [23]:
phraseVecModel = Phrase2Vec.from_word2vec_paths(300,
                                                "data/word2vec/GoogleNews-vectors-negative300.bin",
                                                "example_emoji2vec/emoji2vec.bin")

In [24]:
# mapping from id to emoji
mapping = pk.load(open(params["mapping_file"], 'rb'))
# mapping from emoji to id
inverse_mapping = {v: k for k, v in mapping.items()}

In [25]:
def find_analogous_emoji(emoji_1,
                         emoji_2,
                         emoji_3,
                         top_n,
                         mapping=mapping,
                         inverse_mapping=inverse_mapping,
                         e2v=e2v,
                         model=model):
    similarities = []
    vector = e2v[emoji_1] - e2v[emoji_2] + e2v[emoji_3]
    vector = vector / np.linalg.norm(vector)
    
    for idx in range(len(mapping)):
        emoij_idx_similarity = model.nn.forward(torch.Tensor(vector.reshape(1, -1)), idx).detach().numpy()
        similarities.append(emoij_idx_similarity)
    
    similarities = np.array(similarities)
    n_most_similar_idxs = similarities.argsort(axis=0)[-top_n:][::-1].reshape(-1)
    n_most_similar_emojis = [mapping[emoji_idx] for emoji_idx in n_most_similar_idxs]
    
    str_expression = ' '.join([emoji_1, "-", emoji_2, "+", emoji_3,])
    top_score = similarities[n_most_similar_idxs[0]]
    
    return str_expression, n_most_similar_emojis
        

In [26]:
table = [["Expression", "Closest emojis"],
         [*find_analogous_emoji("🤴", "🚹", "🚺", 3)],
         [*find_analogous_emoji("👑", "🚹", "🚺", 3)],
         [*find_analogous_emoji("👦", "🚹", "🚺", 3)],
         [*find_analogous_emoji("💵", "🇺🇸", "🇬🇧", 3)],
         [*find_analogous_emoji("💵", "🇺🇸", "🇪🇺", 3)],
         [*find_analogous_emoji("💷", "🇬🇧", "🇪🇺", 3)]]

In [27]:
print(tabulate(table, headers="firstrow", tablefmt="rst"))

============  ==================================
Expression    Closest emojis
============  ==================================
🤴 - 🚹 + 🚺  ['🤴', '👸', '👑']
👑 - 🚹 + 🚺  ['👑', '👸', '💎']
👦 - 🚹 + 🚺  ['👦', '👨\u200d🎓', '👨\u200d🏫']
💵 - 🇺🇸 + 🇬🇧  ['💵', '👛', '🤕']
💵 - 🇺🇸 + 🇪🇺  ['💵', '👑', '🦏']
💷 - 🇬🇧 + 🇪🇺  ['💷', '🇪🇺', '🦏']
============  ==================================


Although far from perfect, similarly to the [published](https://arxiv.org/pdf/1609.08359.pdf) results an emoji considered to be "right" is usually within the top 3 examples selected by the model.

Combining Emoji2Vec representations with word2vec embeddings they were trained on also allows to inspect the relationships between word embeddings and emoji embeddings. For example with the `phrase2emoji` function below we can see emojis most similar to an arbitrary phrase.

In [28]:
def phrase2emoji(phrase, 
                 top_n,
                 phraseVecModel=phraseVecModel,
                 mapping=mapping,
                 model=model):
    
    similarities = []
    phrase_vec = phraseVecModel[phrase]
    for idx in range(len(mapping)):
        emoij_idx_similarity = model.nn.forward(torch.Tensor(phrase_vec.reshape(1, -1)), idx).detach().numpy()
        similarities.append(emoij_idx_similarity)
        
    similarities = np.array(similarities)
    n_most_similar_idxs = similarities.argsort(axis=0)[-top_n:][::-1].reshape(-1)
    n_most_similar_emojis = [mapping[emoji_idx] for emoji_idx in n_most_similar_idxs]
    
    return phrase, n_most_similar_emojis
    

In [29]:
table = [["Phrase", "Closest emojis"],
         [*phrase2emoji("funny", 3)],
         [*phrase2emoji("scary", 3)],
         [*phrase2emoji("okay", 3)],
         [*phrase2emoji("crazy", 3)],
         [*phrase2emoji("wild", 3)],
         [*phrase2emoji("afraid", 3)]]

In [30]:
print(tabulate(table, headers="firstrow", tablefmt="rst"))

========  ==========================
Phrase    Closest emojis
========  ==========================
funny     ['👻', '😂', '💩']
scary     ['👻', '🤡', '🦇']
okay      ['👍', '👌', '🉑']
crazy     ['🤪', '🦉', '🥜']
wild      ['🐅', '🤪', '🐖']
afraid    ['🤡', '😷', '\U0001fab3']
========  ==========================


There seems to exist a strong relationship between simple adjectives and emojis. That's great, but is this the whole point of Emoji2Vec? <br>
Of course not. In fact Emoji2Vec authors evaluated their method's performance on a downstream task of sentiment classification on a dataset  by  Kralj   Novak  et  al.  (2015),  which  consists of over 67k English tweets labelled manually for positive, neutral, or negative sentiment. Using Emoji2Vec alongside word embeddings, as opposed to using just word embeddings, yielded an improvement in classification accuracy across all studied datasets. In the same task Emoji2Vec also outperformed an alternative method for emoji representation. <br>
For more details check out the [paper](https://arxiv.org/pdf/1609.08359.pdf). You can also inspect the results of the sentiment classification task in [this notebook](https://github.com/uclmr/emoji2vec/blob/master/TwitterClassification.ipynb).

In [31]:
plutchick = [["Phrase", "Closest emojis"],
         [*phrase2emoji("serenity", 5)],
         [*phrase2emoji("joy", 5)],
         [*phrase2emoji("ecstasy", 5)],
         [*phrase2emoji("love", 5)],
         [*phrase2emoji("acceptance", 5)],
         [*phrase2emoji("trust", 5)],
         [*phrase2emoji("admiration", 5)],
         [*phrase2emoji("submission", 5)],
         [*phrase2emoji("apprehension", 5)],
         [*phrase2emoji("fear", 5)],
         [*phrase2emoji("terror", 5)],
         [*phrase2emoji("awe", 5)],
         [*phrase2emoji("distraction", 5)],
         [*phrase2emoji("surprise", 5)],
         [*phrase2emoji("amazement", 5)],
         [*phrase2emoji("disapproval", 5)],
         [*phrase2emoji("pensiveness", 5)],
         [*phrase2emoji("sadness", 5)],
         [*phrase2emoji("grief", 5)],
         [*phrase2emoji("remorse", 5)],
         [*phrase2emoji("boredom", 5)],
         [*phrase2emoji("disgust", 5)],
         [*phrase2emoji("loathing", 5)],
         [*phrase2emoji("contempt", 5)],
         [*phrase2emoji("annoyance", 5)],
         [*phrase2emoji("anger", 5)],
         [*phrase2emoji("rage", 5)],
         [*phrase2emoji("aggressive", 5)],
         [*phrase2emoji("interest", 5)],
         [*phrase2emoji("anticipation", 5)],
         [*phrase2emoji("vigilance", 5)]]

In [32]:
print(tabulate(plutchick, headers="firstrow", tablefmt="rst"))

============  ==========================================================
Phrase        Closest emojis
============  ==========================================================
serenity      ['😌', '😎', '🕉', '⛪', '🦄']
joy           ['\ufeff😀', '🌈', '👨\u200d👩\u200d👧', '😺', '🤠']
ecstasy       ['⚗', '🍷', '🍄', '🍭', '💊']
love          ['💓', '😍', '💞', '❤', '💝']
acceptance    ['👍', '💯', '👀', '😉', '🔑']
trust         ['🧑\u200d🤝\u200d🧑', '🤑', '👬', '👭', '👫']
admiration    ['🙇\u200d♂️', '🙇', '😻', '😙', '💓']
submission    ['🙇', '🙇\u200d♂️', '🧎\u200d♀️', '🙇\u200d♀️', '🧎\u200d♂️']
apprehension  ['😦', '😱', '😲', '😧', '😟']
fear          ['😱', '🙀', '💉', '😷', '🐍']
terror        ['💣', '🧟\u200d♂️', '🏳', '🇦🇫', '🇪🇹']
awe           ['😮', '🤯', '😱', '😵', '🧨']
distraction   ['🤷', '🤷\u200d♀️', '🤷\u200d♂️', '🙎\u200d♀️', '🧖\u200d♀️']
surprise      ['😦', '😲', '😧', '⁉', '🎆']
amazement     ['😲', '😮', '🙉', '😵', '🤯']
disapproval   ['👎', '😒', '😕', '🙅\u200d♀️', '😠']
pensiveness   ['👽', '🤫', '🎻', '🎵', '😹']
sadness       ['😦', '

In [33]:
 humour = [["Phrase", "Closest emojis"],
         [*phrase2emoji("funny", 20)],
         [*phrase2emoji("laughing", 20)],
         [*phrase2emoji("hysterical", 20)],
         [*phrase2emoji("joke", 20)],
         [*phrase2emoji("sarcasm", 20)],
         [*phrase2emoji("irony", 20)],
         [*phrase2emoji("mischief", 20)],
         [*phrase2emoji("prank", 20)],
         [*phrase2emoji("naughty", 20)]
         
        ]

In [34]:
print(tabulate(humour, headers="firstrow", tablefmt="rst"))

==========  =====================================================================================================================================================
Phrase      Closest emojis
==========  =====================================================================================================================================================
funny       ['👻', '😂', '💩', '😛', '😹', '😜', '😆', '🐵', '🤣', '🤪', '🤡', '🌝', '😉', '😸', '😝', '🙃', '🐒', '🐷', '🐥', '👯\u200d♂️']
laughing    ['🙈', '🤣', '😂', '🤪', '💩', '😜', '😸', '🙆', '🐵', '😆', '👻', '🦗', '🥳', '😛', '😝', '👯\u200d♂️', '🙃', '🤔', '⏹', '🦉']
hysterical  ['🤣', '🙀', '💀', '😆', '🐵', '😜', '🐣', '🥴', '🙃', '😱', '🐊', '🌝', '🙅\u200d♀️', '🤪', '🙈', '👻', '🐒', '💩', '😝', '🤡']
joke        ['🌝', '😸', '💩', '😜', '👻', '🙃', '😂', '🤣', '🐵', '👴', '😉', '😛', '🦗', '🤪', '🐽', '😆', '🙈', '🤡', '😝', '\U0001f9ab']
sarcasm     ['🙃', '🌝', '😏', '🎻', '😛', '💁\u200d♂️', '💁', '💁\u200d♀️', '😜', '🐸', '😼', '💀', '😇', '🖕', '😸', '😆', '🙉', '😉', '😹', '😒']
irony       ['🌝', '😏', '🙃', '🥀', '🤑

In [35]:
success = [["Phrase", "Closest emojis"],
           [*phrase2emoji("luck", 5)],
           [*phrase2emoji("success", 5)],
           [*phrase2emoji("boasting", 5)],
           [*phrase2emoji("achievement", 5)],
           [*phrase2emoji("money", 5)]]
            

In [36]:
print(tabulate(success, headers="firstrow", tablefmt="rst"))

===========  =============================================
Phrase       Closest emojis
===========  =============================================
luck         ['🧚\u200d♂️', '\U0001fa99', '🤞', '🥠', '🎲']
success      ['🥇', '🔑', '⭐', '🀄', '🤹']
boasting     ['😝', '🆕', '➕', '😜', '🦚']
achievement  ['🥇', '🎖', '👨\u200d🎓', '🥉', '🥈']
money        ['🏦', '\U0001fa99', '🤑', '🏧', '🧧']
===========  =============================================


In [37]:
 yesno = [["Phrase", "Closest emojis"],
          [*phrase2emoji("no", 5)],
         [*phrase2emoji("yes", 5)],
         [*phrase2emoji("approval", 5)],
         [*phrase2emoji("support", 5)]]

In [38]:
 print(tabulate(yesno, headers="firstrow", tablefmt="rst"))

========  ===========================================================
Phrase    Closest emojis
========  ===========================================================
no        ['👎', '🆖', '🙅', '🈵', '🙅\u200d♂️']
yes       ['👍', '🉑', '👎', '👩\u200d❤️\u200d👨', '❓']
approval  ['✔', '👀', '👍', '🆗', '👌']
support   ['🤗', '🏳️\u200d🌈', '🧑\u200d💼', '👩\u200d💼', '🏳️\u200d⚧️']
========  ===========================================================


In [39]:
silence = [["Phrase", "Closest emojis"],
          [*phrase2emoji("secret", 5)],
          [*phrase2emoji("discretion", 5)],
          [*phrase2emoji("private", 5)],
          [*phrase2emoji("disguise", 5)],
          [*phrase2emoji("shut up", 5)],
          [*phrase2emoji("silence", 5)]]


In [40]:
print(tabulate(silence, headers="firstrow", tablefmt="rst"))

==========  =================================================
Phrase      Closest emojis
==========  =================================================
secret      ['㊙', '\U0001fab1', '🙊', '🕵️\u200d♂️', '🐽']
discretion  ['🙊', '🙉', '🧑\u200d⚖️', '👑', '👨\u200d⚖️']
private     ['\U0001f978', '🕵️\u200d♀️', '🕵️\u200d♂️', '🏘', '🈲']
disguise    ['\U0001f978', '👺', '\U0001f977', '🐽', '🤺']
shut up     ['🤐', '🈺', '⤴', '\U0001faa0', '🚨']
silence     ['😶', '📣', '🔇', '🎻', '⛪']
==========  =================================================


In [41]:
gossip = [["Phrase", "Closest emojis"],
         [*phrase2emoji("indifference", 5)],
         [*phrase2emoji("gossip", 5)],
         [*phrase2emoji("sassy", 5)],
         [*phrase2emoji("liar", 5)],
         [*phrase2emoji("tattletale", 5)],
         [*phrase2emoji("wierd", 5)],
         [*phrase2emoji("geek", 5)],
         [*phrase2emoji("evil", 5)],
         [*phrase2emoji("life", 5)],
         [*phrase2emoji("death", 5)]]


In [42]:
print(tabulate(gossip, headers="firstrow", tablefmt="rst"))

============  ==============================================================
Phrase        Closest emojis
============  ==============================================================
indifference  ['💅', '😑', '🤷\u200d♂️', '🤷', '🤷\u200d♀️']
gossip        ['🧑\u200d🚒', '👨\u200d🚒', '\U0001fab1', '👩\u200d🚒', '🐸']
sassy         ['💁', '💁\u200d♂️', '👯\u200d♀️', '👯', '👸']
liar          ['🤥', '🐀', '🤦\u200d♂️', '🐥', '🐔']
tattletale    ['\U0001fab1', '🐵', '🐥', '🤥', '🐽']
wierd         ['👻', '🤖', '🐥', '🐣', '🤓']
geek          ['👨\u200d💻', '👩\u200d💻', '👾', '🤓', '🤖']
evil          ['🦹', '👿', '👹', '🦹\u200d♂️', '🦹\u200d♀️']
life          ['🧬', '🌆', '\U0001fac0', '🌉', '🌃']
death         ['⚰', '\U0001faa6', '🐡', '⚱', '😵']
============  ==============================================================


In [43]:
 positive = [["Phrase", "Closest emojis"],
         [*phrase2emoji("warmth", 5)],
         [*phrase2emoji("positive", 5)],
         [*phrase2emoji("relaxation", 5)],
         [*phrase2emoji("calm", 5)],
         [*phrase2emoji("stress", 5)],
         [*phrase2emoji("storm", 5)],
         [*phrase2emoji("vacation", 5)]]

In [44]:
print(tabulate(positive, headers="firstrow", tablefmt="rst"))

==========  =================================================
Phrase      Closest emojis
==========  =================================================
warmth      ['😊', '😸', '❄', '🤗', '🌷']
positive    ['⤴', '🙂', '📈', '😌', '🌞']
relaxation  ['🧘\u200d♀️', '🕯', '🧘', '🏌️\u200d♂️', '💆\u200d♂️']
calm        ['🕉', '😌', '☮', '🧊', '😶\u200d🌫️']
stress      ['🧖\u200d♀️', '💦', '🧖', '🧖\u200d♂️', '🙀']
storm       ['🌀', '⛈', '🌨', '🌪', '🌧']
vacation    ['🌴', '🏖', '🕶', '🚤', '🚵\u200d♂️']
==========  =================================================


In [45]:
judge = [["Phrase", "Closest emojis"],
        [*phrase2emoji("skepticism", 5)],
        [*phrase2emoji("questioning", 5)],
        [*phrase2emoji("judgement", 5)]]

In [46]:
print(tabulate(judge, headers="firstrow", tablefmt="rst"))

===========  ===================================================
Phrase       Closest emojis
===========  ===================================================
skepticism   ['😲', '🤷\u200d♂️', '😕', '🤷', '😦']
questioning  ['⁉', '🧐', '🤔', '👨\u200d❤️\u200d👨', '🧑\u200d⚖️']
judgement    ['🆎', '🔝', '📷', '💄', '🟩']
===========  ===================================================


In [47]:
movement = [["Phrase", "Closest emojis"],
        [*phrase2emoji("late", 5)],
        [*phrase2emoji("hurry", 5)],
        [*phrase2emoji("escape", 5)],
        [*phrase2emoji("relief", 5)],
        [*phrase2emoji("movement", 5)],
        [*phrase2emoji("descending", 5)]]

In [48]:
print(tabulate(movement, headers="firstrow", tablefmt="rst"))

==========  ===================================================
Phrase      Closest emojis
==========  ===================================================
late        ['🚑', '🦽', '🦼', '⏲', '🏃']
hurry       ['🏃', '🏃\u200d♀️', '🏃\u200d♂️', '⏩', '⏫']
escape      ['\U0001fa9d', '🧗', '🧗\u200d♀️', '🧗\u200d♂️', '⏏']
relief      ['😮\u200d💨', '😅', '😂', '😌', '🏃\u200d♀️']
movement    ['✊', '♓', '🏴\u200d☠️', '\U0001faa7', '🕺']
descending  ['⛷', '🏂', '🪂', '🧗\u200d♂️', '⏏']
==========  ===================================================


In [49]:
friend = [["Phrase", "Closest emojis"],
        [*phrase2emoji("celebration", 5)],
         [*phrase2emoji("friendship", 5)],
          [*phrase2emoji("family", 5)],
          [*phrase2emoji("fascination", 5)]]
         

In [50]:
print(tabulate(friend, headers="firstrow", tablefmt="rst"))

===========  ====================================================
Phrase       Closest emojis
===========  ====================================================
celebration  ['🎏', '🍸', '🍻', '🧧', '🎁']
friendship   ['👬', '💕', '❤', '👭', '👩\u200d❤️\u200d💋\u200d👨']
family       ['🏠', '👪', '🤰', '👩\u200d👦', '👨\u200d👧']
fascination  ['☄', '😍', '💓', '💞', '😻']
===========  ====================================================


In [51]:
fit = [["Phrase", "Closest emojis"],
    [*phrase2emoji("sport", 5)],
    [*phrase2emoji("exercise", 5)],
    [*phrase2emoji("diet", 5)],
    [*phrase2emoji("fat", 5)],
    [*phrase2emoji("appearence", 5)]]
         

In [52]:
print(tabulate(fit, headers="firstrow", tablefmt="rst"))

==========  ===========================================================
Phrase      Closest emojis
==========  ===========================================================
sport       ['🏑', '⚾', '🎽', '🏆', '🥍']
exercise    ['⏱', '🏊', '👟', '🚴\u200d♀️', '🏇']
diet        ['🥒', '🥗', '🥬', '🥑', '🌽']
fat         ['\U0001f9ac', '🤰', '🐄', '🐳', '🐖']
appearence  ['💇', '💇\u200d♂️', '💇\u200d♀️', '👩\u200d🦰', '🤵\u200d♂️']
==========  ===========================================================


In [53]:
politics = [["Phrase", "Closest emojis"],
    [*phrase2emoji("race", 5)],
    [*phrase2emoji("age", 5)],
    [*phrase2emoji("disability", 5)],
    [*phrase2emoji("politics", 5)],
    [*phrase2emoji("religion", 5)]]

In [54]:
 print(tabulate(politics, headers="firstrow", tablefmt="rst"))

==========  =============================================================
Phrase      Closest emojis
==========  =============================================================
race        ['🐎', '🎌', '🏇', '🥈', '🥉']
age         ['🧑\u200d🦳', '👴', '🧑\u200d🦼', '🧓', '🌱']
disability  ['👩\u200d👩\u200d👧\u200d👧', '🦼', '🏥', '🦽', '🧑\u200d⚕️']
politics    ['\U0001faa7', '🇱🇧', '🇩🇴', '🇰🇵', '🇸🇾']
religion    ['🕍', '🛐', '🕉', '📿', '✡']
==========  =============================================================


In [55]:
timeofyear = [["Phrase", "Closest emojis"],
        [*phrase2emoji("halloween", 5)],
        [*phrase2emoji("christmas", 5)],
        [*phrase2emoji("easter", 5)],
        [*phrase2emoji("new year", 5)],
        [*phrase2emoji("ramadan", 5)],
        [*phrase2emoji("winter", 5)],
        [*phrase2emoji("summer", 5)],
        [*phrase2emoji("spring", 5)],
        [*phrase2emoji("autumn", 5)],
        [*phrase2emoji("day", 5)],
        [*phrase2emoji("night", 5)],
        [*phrase2emoji("light", 5)],
        [*phrase2emoji("dark", 5)]]

In [56]:
print(tabulate(timeofyear, headers="firstrow", tablefmt="rst"))

=========  ============================================
Phrase     Closest emojis
=========  ============================================
halloween  ['⚰', '🎃', '🤡', '👻', '🌕']
christmas  ['🤶', '🎅', '☃', '🦃', '🎁']
easter     ['🥚', '🐥', '🐤', '🐇', '🌸']
new year   ['🎇', '🎍', '🌱', '🐲', '🎈']
ramadan    ['👳\u200d♂️', '👳', '👳\u200d♀️', '🇾🇪', '🥮']
winter     ['🌨', '🧤', '🧥', '🧣', '⛄']
summer     ['🩳', '🩱', '🌞', '👙', '🍃']
spring     ['🍃', '🌱', '🌿', '🌼', '🕓']
autumn     ['🍂', '🍃', '🍁', '🌸', '🥮']
day        ['🕗', '🕧', '🕦', '🕠', '🕡']
night      ['🕔', '✴', '🌇', '🕟', '🕐']
light      ['☀', '🏮', '🔦', '💡', '🟡']
dark       ['🖤', '🔦', '🥕', '😨', '🌑']
=========  ============================================


In [57]:
job = [["Phrase", "Closest emojis"],
    [*phrase2emoji("career", 5)],
    [*phrase2emoji("student", 5)],
    [*phrase2emoji("learning", 5)],
    [*phrase2emoji("knowledge", 5)],
    [*phrase2emoji("hard work", 5)],
    [*phrase2emoji("repetition", 5)],
    [*phrase2emoji("amateur", 5)],
    [*phrase2emoji("job", 5)]]

In [58]:
 print(tabulate(job, headers="firstrow", tablefmt="rst"))

==========  ======================================================================
Phrase      Closest emojis
==========  ======================================================================
career      ['👔', '⭐', '♍', '🌉', '🤸\u200d♂️']
student     ['👨\u200d🎓', '👩\u200d🎓', '🧪', '🧑\u200d🎓', '🥋']
learning    ['👨\u200d🎓', '🧑\u200d🎓', '💡', '👨\u200d🏫', '🤯']
knowledge   ['👨\u200d🏫', '🧑\u200d🏫', '🤓', '🧝', '🧝\u200d♂️']
hard work   ['⚙', '🏋️\u200d♀️', '⚒', '👩\u200d🏭', '🔨']
repetition  ['🔄', '🔁', '🧑\u200d🏭', '😛', '🤹']
amateur     ['🧑\u200d🍳', '🧑\u200d🎤', '👨\u200d🍳', '👨\u200d🎨', '👨\u200d🎤']
job         ['🙆\u200d♂️', '👷\u200d♂️', '👞', '🏢', '🏭']
==========  ======================================================================


In [59]:
plutchick2 = [["Phrase", "Closest emojis"],
         [*phrase2emoji("serenity", 20)],
         [*phrase2emoji("joy", 20)],
         [*phrase2emoji("ecstasy", 20)],
         [*phrase2emoji("love", 20)],
         [*phrase2emoji("acceptance", 20)],
         [*phrase2emoji("trust", 20)],
         [*phrase2emoji("admiration", 20)],
         [*phrase2emoji("submission", 20)],
         [*phrase2emoji("apprehension", 20)],
         [*phrase2emoji("fear", 20)],
         [*phrase2emoji("terror", 20)],
         [*phrase2emoji("awe", 20)],
         [*phrase2emoji("distraction", 20)],
         [*phrase2emoji("surprise", 20)],
         [*phrase2emoji("amazement", 20)],
         [*phrase2emoji("disapproval", 20)],
         [*phrase2emoji("pensiveness", 20)],
         [*phrase2emoji("sadness", 20)],
         [*phrase2emoji("grief", 20)],
         [*phrase2emoji("remorse", 20)],
         [*phrase2emoji("boredom", 20)],
         [*phrase2emoji("disgust", 20)],
         [*phrase2emoji("loathing", 20)],
         [*phrase2emoji("contempt", 20)],
         [*phrase2emoji("annoyance", 20)],
         [*phrase2emoji("anger", 20)],
         [*phrase2emoji("rage", 20)],
         [*phrase2emoji("aggressive", 20)],
         [*phrase2emoji("interest", 20)],
         [*phrase2emoji("anticipation", 20)],
         [*phrase2emoji("vigilance", 20)]]

In [60]:
print(tabulate(plutchick2, headers="firstrow", tablefmt="rst"))

============  =============================================================================================================================================================================================================
Phrase        Closest emojis
============  =============================================================================================================================================================================================================
serenity      ['😌', '😎', '🕉', '⛪', '🦄', '🌈', '🏵', '🦋', '🌷', '🍷', '🧖\u200d♀️', '☮', '🌼', '🌹', '😊', '🚣\u200d♂️', '🧝', '😶\u200d🌫️', '🏌️\u200d♀️', '🕊']
joy           ['\ufeff😀', '🌈', '👨\u200d👩\u200d👧', '😺', '🤠', '😭', '🎆', '🧧', '🥳', '🎄', '😊', '😫', '👨\u200d👨\u200d👧', '🎵', '👨\u200d👩\u200d👦', '🙆\u200d♂️', '🥰', '👩\u200d👦', '👩\u200d👩\u200d👦', '😝']
ecstasy       ['⚗', '🍷', '🍄', '🍭', '💊', '👨\u200d👨\u200d👧', '🍸', '🐉', '🎆', '🥴', '🍯', '💫', '👩\u200d🎨', '☠', '🙆\u200d♂️', '👨\u200d👩\u200d👧', '👨\u200d❤️\u200d💋\u200d👨', '🌫', '🥃', '🧜']
love       

In [93]:
sentence = [["Phrase", "Closest emojis"],
    [*phrase2emoji("I", 1)],
    [*phrase2emoji("walked", 1)],
    [*phrase2emoji("the", 1)],
    [*phrase2emoji("dog", 1)],
    [*phrase2emoji("in", 1)],
    [*phrase2emoji("the", 1)],
    [*phrase2emoji("park", 1)]]

In [94]:
print(tabulate(sentence, headers="firstrow", tablefmt="rst"))

========  ================
Phrase    Closest emojis
========  ================
I         ['🉑']
walked    ['🚶\u200d♀️']
the       ['👆']
dog       ['🐶']
in        ['💑']
the       ['👆']
park      ['⛲']
it        ['🤙']
was       ['🤷\u200d♂️']
great     ['🥳']
========  ================


In [77]:
sentence2 = [["Phrase", "Closest emojis"],
    [*phrase2emoji("I", 1)],
    [*phrase2emoji("love", 1)],
    [*phrase2emoji("you", 1)]]

In [78]:
print(tabulate(sentence2, headers="firstrow", tablefmt="rst"))

========  ================
Phrase    Closest emojis
========  ================
I         ['🉑']
love      ['💓']
you       ['👷']
========  ================
